Before this, you should have downloaded and unzipped all the data from kaggle.
Be sure to update your working directory to get the data all formatted.

This is directly from https://www.kaggle.com/yhn112/resnet18-baseline-pytorch-ignite/notebook

In [21]:
import pandas as pd
from PIL import Image
import torch
import torch.utils.data as D
from torchvision import models, transforms as T
from sklearn.model_selection import train_test_split

Be sure to update path_data

In [22]:
path_data = 'D:\\Data\\DeepLearning\\recursion-cellular-image-classification'
device = 'cuda'
batch_size = 32
torch.manual_seed(0)

In [23]:
class ImagesDS(D.Dataset):
    def __init__(self, df, img_dir, mode='train', site=1, channels=[1,2,3,4,5,6]):
        self.records = df.to_records(index=False)
        self.channels = channels
        self.site = site
        self.mode = mode
        self.img_dir = img_dir
        self.len = df.shape[0]

    @staticmethod
    def _load_img_as_tensor(file_name):
        with Image.open(file_name) as img:
            return T.ToTensor()(img)

    def _get_img_path(self, index, channel):
        experiment, well, plate = self.records[index].experiment, self.records[index].well, self.records[index].plate
        return '/'.join([self.img_dir,self.mode,experiment,f'Plate{plate}',f'{well}_s{self.site}_w{channel}.png'])

    def __getitem__(self, index):
        paths = [self._get_img_path(index, ch) for ch in self.channels]
        img = torch.cat([self._load_img_as_tensor(img_path) for img_path in paths])
        if self.mode == 'train':
            return img, int(self.records[index].sirna)
        else:
            return img, self.records[index].id_code

    def __len__(self):
        return self.len

In [24]:
df = pd.read_csv(path_data+'/train.csv')
df_train, df_val = train_test_split(df, test_size = 0.025, random_state=42)
df_test = pd.read_csv(path_data+'/test.csv')

In [25]:
ds = ImagesDS(df_train, path_data, mode='train')
ds_val = ImagesDS(df_val, path_data, mode='train')
ds_test = ImagesDS(df_test, path_data, mode='test')